### Table of Contents

- [1. Introduction to the 2D Robot Arm](#1-introduction-to-the-2d-robot-arm)
    - [Exercise 1.3.1](#exercise-131)
- [2. Animating the Robot Arm](#2-animating-the-robot-arm)
    - [Exercise 2.1.1](#exercise-211)
    - [Exercise 2.1.2](#exercise-212)
    - [2.2 Reflection](#22-reflection)
- [3. Discrete-Time State Space Model and Lifted System Representation](#3-discrete-time-state-space-model-and-lifted-system-representation)
    - [Exercise 3.2.1](#exercise-321)
    - [Exercise 3.2.2](#exercise-322)
    - [3.3 Reflection](#33-reflection)
- [4. Iterative Learning Control Law](#4-iterative-learning-control-law)
    - [Exercise 4.3.1](#exercise-431)
    - [Exercise 4.3.2](#exercise-432)
    - [Exercise 4.3.3](#exercise-433)
    - [Exercise 4.3.4](#exercise-434)
    - [Exercise 4.3.5](#exercise-435)
    - [4.4 Reflection](#44-reflection)

### Dependencies

You can run the following cell to install all the libraries we'll be using throughout this notebook.

In [ ]:
%pip install numpy plotly scipy sympy

# Iterative Learning Control for a 2D Robot Arm

## 1. Introduction to the 2D Robot Arm

In this assignment, we will implement Iterative Learning Control (ILC) for a two-link planar robot arm. Before we dive into the control algorithm, let's understand the basics of our robot arm.

### 1.1 The 2D Robot Arm

A two-link planar robot arm consists of two rigid links connected by revolute joints. It operates in a 2D plane and is described by the following parameters:

- $l_1$: Length of the first link
- $l_2$: Length of the second link
- $\theta_1$: Angle of the first link relative to the horizontal axis
- $\theta_2$: Angle of the second link relative to the first link

The configuration of the arm can be fully described by the joint angles $\theta_1$ and $\theta_2$.

### 1.2 End-Effector

The end-effector is the point of interest at the end of the robot arm. In our case, it's the tip of the second link. The position of the end-effector in Cartesian coordinates $(x, y)$ can be calculated using forward kinematics.

Let's assume the position of the first joint $(x_1, y_1)$ is just the origin $(0, 0)$. We then can calculate the position of the second joint $(x_2, y_2)$ as:

$x_2 = x_1 + l_1 \cos(\theta_1)$ - the x-coordinate of the first joint.  
$y_2 = y_1 + l_1 \sin(\theta_1)$ - the y-coordinate of the first joint.

Considering now the second link, the position of the end-effector $(x, y)$ can be calculated as follows:  

$x = x_2 + l_2 \cos(\theta_1 + \theta_2)$  
$y = y_2 + l_2 \sin(\theta_1 + \theta_2)$

### 1.3 Drawing the Robot Arm

Our first task is to create a function that draws the 2D robot arm given its joint angles. This will help us visualize the arm's movement as we develop our control algorithm.

#### Exercise 1.3.1
Implement the method `joint_positions()` for the class `RobotArm`. A `RobotArm` object represents our robot arm, with the `l_1` and `l_2` attributes of the `RobotArm` being the link lengths and the `theta_1` and `theta_2` attributes - the joint angles.
The `joint_positions()` method should return a list of `Point` objects, corresponding to the cartesian coordinates of the joints and the end-effector, with the end-effector coordinates at the very end of the list. 

Once this is done, the `draw()` method will be able to produce a plot of the robot arm in the current configuration. More precisely, the `draw()` method will:

1. Calculate the positions of the joints and the end-effector using the `joint_positions()` method.
2. Plot the links as lines.
3. Mark the joints and end-effector with points.
4. Set appropriate axis limits so that the robot isn't clipped.

> Hint: You can use `np.cos` and `np.sin` to calculate the cosine and sine values of an angle.

In [ ]:
import numpy as np
from dataclasses import dataclass, KW_ONLY, field
from typing import Final
from helpers import RobotArmMixin, Point


@dataclass
class RobotArm(RobotArmMixin):
    _: KW_ONLY  # to prevent positional instantiation, since it leads to bugs
    name: Final[str] = "Helpful Roboto :)"
    l_1: Final[float]
    l_2: Final[float]
    theta_1: float
    theta_2: float

    rotations: int = field(default=0, init=False)

    def joint_positions(self) -> list[Point]:
        # {% if exercise['1.3.1'].solution %}
        # TODO: Review me!
        x_0, y_0 = 0, 0
        x_1, y_1 = (
            x_0 + self.l_1 * np.cos(self.theta_1),
            y_0 + self.l_1 * np.sin(self.theta_1),
        )
        x, y = (
            x_1 + self.l_2 * np.cos(self.theta_1 + self.theta_2),
            y_1 + self.l_2 * np.sin(self.theta_1 + self.theta_2),
        )

        return [Point(x_0, y_0), Point(x_1, y_1), Point(x, y)]
        # {% else %}
        # TODO: Implement me!
        raise NotImplementedError("Not implemented!")
        # {% endif %}

Now let's use the `draw()` method to draw the arm in a few different configurations. Here is what our robot arms look like.

In [ ]:
RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=0, theta_2=0).draw()
_ = RobotArm(
    name="Jeff", l_1=0.5, l_2=1, theta_1=np.pi / 2, theta_2=-np.pi / 2
).draw()  # We store the result so it isn't double-displayed

You can also rotate the arms by changing the joint angles.

In [ ]:
roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=0, theta_2=0)
roberto.draw()

roberto.rotate_to(np.pi / 2, -np.pi / 2)
_ = roberto.draw()

## 2. Animating the Robot Arm

Now that we can draw static positions of our robot arm, let's bring it to life with animation. This will help us visualize how the arm moves over time, as well as how our control logic improves throughout the learning process. It's
also fun to watch!

### 2.1 Creating an Animation

We'll create an animation of the robot arm moving through a sequence of positions. This will involve:

1. Generating a sequence of joint angles.
2. Using our `RobotArm.draw()` method for each frame.
3. Combining these frames into an animation.

#### Exercise 2.1.1
Implement the function `animation_frames_for(robot, joint_angles)` that takes our robot and a sequence of joint angle pairs $(\theta_1, \theta_2)$ and produces a list of frames of the robot arm moving.

Tips:
- You can use `robot.draw(show=False)` to generate a frame, without actually showing it in the output.
- The `trace=...` argument for the `robot.draw` method can be used to visualize the movement of the end-effector. 
- The value for the `trace` argument should be a list of points, representing the positions of the end-effector over time. 
- It may be a good idea to gradually build up the end-effector trace, while you iterate over the joint angles.
- Don't forget to return the frames as the result of the function!

Afterwards, the `animate(robot, joint_angles)` function will:

1. Create the animation frames using the `animation_frames_for` method.
2. Combine these frames into a single animation.
3. Add a button to the top-left of the plot to play the animation.

You will definitely have to verify your implementation in the next exercise, however, feel free to run the `animate()` function on your own now if you want to check your solution.

In [ ]:
import plotly.graph_objects as go
from helpers import JointAngles


def animation_frames_for(
    robot: RobotArm, joint_angles: list[JointAngles]
) -> list[go.Figure]:
    # {% if exercise['2.1.1'].solution %}
    # TODO: Review me!
    # {{ exercise['2.1.1'].comment }}
    end_effector_positions: list[Point] = []
    frames: list[go.Figure] = []

    for theta_1, theta_2 in joint_angles:
        robot.rotate_to(theta_1, theta_2)

        end_effector_positions.append(robot.end_effector_position())
        frames.append(robot.draw(show=False, trace=end_effector_positions))

    return frames
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")
    # {% endif %}

You'll use the function defined below in the next exercise.

In [ ]:
from helpers import animate as animation_helper


def animate(
    robot: RobotArm,
    joint_angles: list[JointAngles],
    *,
    align_starting_position: bool = False,
    subtitle: str = "",
) -> None:
    """Animates the robot arm moving through the given joint angles.

    Args:
        robot: The robot arm to animate.
        joint_angles: The joint angles to animate the robot arm through.
        align_starting_position: Whether to align the starting position of the robot arm with the first joint angle.
        subtitle: The subtitle of the animation.

    Example:
        if you want to show the robot arm moving through the configurations:

        (0, 0) -> (pi/6, 0) -> (pi/5, 0) -> (pi/4, pi/6) -> (pi/3, pi/6) -> (pi/2, pi/6)

        You can do that with the following code:

        ```python
        robot = ... # Some robot arm
        joint_angles = [
            JointAngles(0, 0),
            JointAngles(pi/6, 0),
            JointAngles(pi/5, 0),
            JointAngles(pi/4, pi/6),
            JointAngles(pi/3, pi/6),
            JointAngles(pi/2, pi/6),
        ]

        animate(robot, joint_angles)
        # This will animate the robot arm moving through the configurations.
        ```

        This animation would look quite ugly though. To make a nice smooth animation, you should use
        much more intermediate configurations (e.g. 100).
    """
    animation_helper(
        robot=robot,
        joint_angles=joint_angles,
        animation_frames_for=animation_frames_for,
        align_starting_position=align_starting_position,
        subtitle=subtitle,
    )


#### Exercise 2.1.2
Use the `animate` method provided above to create animations for the following scenarios:

a) The arm moving from a straight vertical position to a horizontal position.  
b) The arm moving from all the way to the left to all the way to the right.  
c) The second link tracing a circular path with the end-effector (first link does not move).  
d) [Challenge] The end-effector tracing a figure 8 path (doesn't have to be perfect).  
e) [Optional] Anything else you want to try!

> Hint: For such motions, you would typically need to use inverse kinematics to calculate the required joint angles, but these cases are simple enough to do by trial and error.

> Better Hint: You can use `np.linspace` to generate a sequence of angles between two configurations.

In [ ]:
# It's a good idea to import pi from numpy, since we're gonna use it a lot
from numpy import pi as π, linspace, sin

# {% if exercise['2.1.2'].solution %}
# TODO: Review me!
# a) Vertical to horizontal
roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=π / 2, theta_2=0)
vertical_to_horizontal: list[JointAngles] = [
    JointAngles(theta_1, 0) for theta_1 in linspace(π / 2, 0, 200)
]
animate(roberto, vertical_to_horizontal)

# b) Left to right
roberto.rotate_to(theta_1=π, theta_2=0)
left_to_right: list[JointAngles] = [
    JointAngles(theta_1, 0) for theta_1 in linspace(π, 0, 200)
]
animate(roberto, left_to_right)

# c) Circle
roberto.rotate_to(theta_1=0, theta_2=0)
second_link_rotation: list[JointAngles] = [
    JointAngles(0, theta_2) for theta_2 in linspace(0, -2 * π, 200)
]
animate(roberto, second_link_rotation)

# d) Figure 8 (a bit ugly)
roberto.rotate_to(theta_1=0, theta_2=π / 2)
t = linspace(0, 4 * π, 200)
theta_1s = π / 8 * sin(t)
theta_2s = π / 2 + π / 4 * sin(2 * t)
theta_sequence: list[JointAngles] = JointAngles.combining(theta_1s, theta_2s)
animate(roberto, theta_sequence)
# {% else %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")
# {% endif %}


### 2.2 Reflection

After creating these animations, consider the following questions:

1. How does the motion of the end-effector relate to the changes in joint angles?
2. Are there any configurations where small changes in joint angles result in large movements of the end-effector?
3. Can you identify any limitations in the arm's range of motion?

As mentioned before, problems like finding a sequence of joint angles to trace a desired trajectory are typically solved using inverse kinematics, either analytically or numerically. We do not
care about how inverse kinematics problems are solved right now, but it's good to know there is a systematic way to do it.

We provide you with the function `trajectory_joint_angles_for` that generates joint angles for the arm to trace the given trajectory. You can how this function can be used, as well as the
resulting animation in the cell below.

> Hint: You can also make up your own trajectories and try tracing them too!

In [ ]:
from helpers import trajectory_joint_angles_for

roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=π / 2, theta_2=0)
square_trajectory: list[Point] = (
    [Point(1, y) for y in linspace(0, 1, 50)]
    + [Point(x, 1) for x in linspace(1, -1, 100)]
    + [Point(-1, y) for y in linspace(1, -1, 100)]
    + [Point(x, -1) for x in linspace(-1, 1, 100)]
    + [Point(1, y) for y in linspace(-1, 0, 50)]
)
animate(
    roberto,
    joint_angles=trajectory_joint_angles_for(
        roberto, square_trajectory, starting_guess=JointAngles(-π / 2, π / 2)
    ),
    align_starting_position=True,
)


wiggly_diamond_trajectory: list[Point] = (
    [Point(1 - t, t + 1 / 4 * sin(4 * π * t)) for t in linspace(0, 1, 100)]
    + [Point(t - 1, t + 1 / 4 * sin(4 * π * t)) for t in linspace(1, 0, 100)]
    + [Point(t - 1, -t - 1 / 4 * sin(4 * π * t)) for t in linspace(0, 1, 100)]
    + [Point(1 - t, -t - 1 / 4 * sin(4 * π * t)) for t in linspace(1, 0, 100)]
)
animate(
    roberto,
    joint_angles=trajectory_joint_angles_for(
        roberto, wiggly_diamond_trajectory, starting_guess=JointAngles(-π / 2, π / 2)
    ),
    align_starting_position=True,
)

## 3. Discrete-Time State Space Model and Lifted System Representation

We can now put Iterative Learning Control into the picture. The goal is to **learn** the ideal torque that should be used to control the second joint. To make
it interesting, we'll be assuming there is some noise impacting the actual angle $\theta_2$ the second joint takes on. 

### 3.1 Discrete-Time State Space Model

For our 2D robot arm, we'll use a simplified discrete-time state space model. The state vector consists of the second joint angle and the corresponding angular velocity, and the input vector consists of the torque applied to the second joint.

The discrete-time state space model is given by:

$x[k+1] = Ax[k] + bu[k]$  
$y[k] = cx[k]$

Where:
- $x[k] = [\theta_2, \omega_2]^T$ is the state vector.
- $u[k] = \tau_2$ is the input.
- $y[k] = \theta_2$ is the output.

The system matrices are:

$A = \begin{bmatrix}
1 & \Delta t \\
0 & 1 - \frac{B}{J} \Delta t
\end{bmatrix}$

$b = \begin{bmatrix}
0 \\
\frac{\Delta t}{J}
\end{bmatrix}$

$c = \begin{bmatrix}
1 & 0
\end{bmatrix}$

Where:
- $\Delta t$ is the sampling time.
- $J$ is the moment of inertia of the second joint.
- $B$ is the damping coefficient of the second joint.

These are just parameters that will be provided later.

### 3.2 Lifted System Representation

In Iterative Learning Control, we consider the system behavior over a finite time horizon of $N$ samples. The lifted system representation allows us to describe the input-output relationship over this entire horizon as a single linear equation.

The lifted system representation is given by:

$\mathbf{y_j} = \mathbf{y_0} + \mathbf{G}\mathbf{u_j} + \mathbf{v_j}$

Where:
- $\mathbf{u_j} = [u_j[0], u_j[1], ..., u_j[N-1]]^T \in \mathbb{R}^N$ is the stacked input vector.
- $\mathbf{v_j} = [v_j[0], v_j[1], ..., v_j[N-1]]^T \in \mathbb{R}^N$ is the disturbance vector.
- $\mathbf{y_0} = [y_0[m], y_0[m+1], ..., y_0[m+N-1]]^T \in \mathbb{R}^N$ is the stacked initial output vector.
- $\mathbf{y_j} = [y_j[m], y_j[m+1], ..., y_j[m+N-1]]^T \in \mathbb{R}^N$ is the stacked output vector over the horizon.

$
\mathbf{G} = \begin{bmatrix}
g[m] & 0 & 0 & \cdots & 0 \\
g[m+1] & g[m] & 0 & \cdots & 0 \\
g[m+2] & g[m+1] & g[m] & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
g[m + N-1] & g[m + N-2] & g[m + N-3] & \cdots & g[m]
\end{bmatrix} \in \mathbb{R}^{N \times N}
$ is the lifted system matrix.

In our case, the impulse response $g[k]$ can be calculated as:

$g[k] = \begin{cases} 0 & \text{for k = 0} \\ c A^{k-1} b & \text{for k = 1, 2, ...} \end{cases}$

$m$ represents the "shift" between the input and output sequences and is defined as $m = r + l$, where $r$ is the **relative degree** and $l$ is the
**additional time delay**.

In addition, once you have the shift $m$, the initial output values $y_0[k]$ (elements of $\mathbf{y_0}$) can be calculated as:

$y_0[k] = c A^{k} x_0 \quad \text{ for } k = m, m + 1, ..., m + N - 1$

where $x_0 := x[0]$ is the initial state vector.

#### Exercise 3.2.1

Implement a function `LiftedSystem.of(A, b, c, N, delay, x_0)` that takes the system matrices $A$, $b$, $c$, the horizon $N$, the delay `delay`, and the initial state vector $x_0$ as `x_0`, and returns the lifted system description, containing:
- The shift $m$.
- The lifted system matrix $\mathbf{G}$.
- The initial output vector $\mathbf{y_0}$.

Afterwards, implement the method `response(u, v)` for the `LiftedSystem` class. This method should calculate the output response $\mathbf{y_j}$ to a given input $\mathbf{u_j}$ and disturbance $\mathbf{v_j}$.

> Hint: It's a good idea to use the matrix functionality of the `numpy` library to perform computations, such as:
> - Matrix creation: `np.array([[1, 2], [3, 4]])` for an arbitrary matrix, `np.zeros((2, 2))` for a zero matrix, and `np.eye(2)` for an identity matrix.
> - Matrix transposition: `A.T` (where `A` is your matrix).
> - Vector-matrix multiplication: `c @ A` or `A @ b` (where `c` and `b` are vectors and `A` is a matrix).  
> &nbsp;

> Another Hint: It's also a good idea to start by calculating the relative degree $r$, which is equal to the first index $k$ for which $g[k] \neq 0$.

> Last Hint: Taking matrix powers efficiently can get a bit tricky, so try using the `power(k)` method of the provided `MatrixPowers` class. An example can be seen in the comments below.

In [ ]:
from helpers import LiftedSystemMixin, MatrixPowers, Matrix, Vector, pretty_latex


MAX_RELATIVE_DEGREE = 1000  # This is just a big number to avoid infinite loops

# To help you get started, we broke down the problem into smaller parts for you.
# Regarding the `MatrixPowers` objects, it's just a wrapper to easily and efficiently
# calculate the powers of a matrix.
# Example usage: `A_squared = A.power(2)`


def impulse_response(k: int, *, A: MatrixPowers, b: Vector, c: Vector) -> float:
    """This function calculates the impulse response of a system for a given k (i.e. g[k])."""
    # {% if exercise['3.2.1'].solution %}
    # TODO: Review me!
    return c.T @ A.power(k - 1) @ b if k > 0 else 0  # type: ignore
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")
    # {% endif %}


def relative_degree_of(*, A: MatrixPowers, b: Vector, c: Vector) -> int:
    """This function calculates the relative degree of a system (i.e. r)."""
    # {% if exercise['3.2.1'].solution %}
    # TODO: Review me!
    # Although one may simply search for a k that makes g(k) != 0, it's a good idea
    # to limit the search to avoid an infinite loop.
    for k in range(MAX_RELATIVE_DEGREE):
        g = impulse_response(k, A=A, b=b, c=c)

        if g != 0:
            return k

    # If we reach this point, it means we couldn't find a relative degree. Something is wrong.
    raise RuntimeError("Could not calculate the relative degree.")
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")
    # {% endif %}


def lifted_system_matrix_of(
    *, A: MatrixPowers, b: Vector, c: Vector, N: int, m: int
) -> Matrix:
    """This function generates the lifted system matrix (i.e. G)."""

    # {% if exercise['3.2.1'].solution %}
    # TODO: Review me!
    # This is a small wrapper function to make the matrix generation syntax easier to read.
    def g(k) -> float:
        return impulse_response(k, A=A, b=b, c=c)

    return np.array(
        [[g(m + i - j) if j <= i else 0 for j in range(N)] for i in range(N)]
    )
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")

    # # You can do something like this to make your code more readable:
    # def g(k) -> float:
    #     return impulse_response(k, A=A, b=b, c=c)
    #
    # # Now you can use `g` to calculate the impulse response at a given k, like this:
    # ... = g(3)  # If you want to calculate g[3]
    # {% endif %}


def initial_output_of(
    *, A: MatrixPowers, c: Vector, x_0: Vector, N: int, m: int
) -> Vector:
    """This function calculates the initial output of a system (i.e. y_0)."""
    # {% if exercise['3.2.1'].solution %}
    # TODO: Review me!
    return np.array([c.T @ A.power(m + k) @ x_0 for k in range(N)])
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")
    # {% endif %}


@dataclass(frozen=True)
class LiftedSystem(LiftedSystemMixin):
    _: KW_ONLY  # Prevents positional arguments.
    m: int
    G: Matrix
    y_0: Vector

    @staticmethod
    def of(
        *, A: Matrix, b: Vector, c: Vector, N: int, delay: int = 0, x_0: Vector
    ) -> "LiftedSystem":
        # {% if exercise['3.2.1'].solution %}
        # TODO: Review me!
        A_powers = MatrixPowers(A)
        r = relative_degree_of(A=A_powers, b=b, c=c)
        m = r + delay
        G = lifted_system_matrix_of(A=A_powers, b=b, c=c, N=N, m=m)
        y_0 = initial_output_of(A=A_powers, c=c, x_0=x_0, N=N, m=m)

        return LiftedSystem(m=m, G=G, y_0=y_0)
        # {% else %}
        # TODO: Implement me!
        A_powers = MatrixPowers(A)

        raise NotImplementedError("Not implemented!")

        # In the end, you can create a `LiftedSystem` object like this:
        # return LiftedSystem(m=m, G=G, y_0=y_0)
        # {% endif %}

    def response(self, u: Vector, v: Vector) -> Vector:
        # {% if exercise['3.2.1'].solution %}
        # TODO: Review me!
        return self.y_0 + self.G @ u + v
        # {% else %}
        # TODO: Implement me!
        raise NotImplementedError("Not implemented!")
        # {% endif %}


#### Exercise 3.2.2
Using the system matrices provided above and a horizon of $N = 100$, construct the lifted system matrix G. You can then use the `plot_responses()` function and compare the impulse responses
calculated using the lifted system representation and the state space representation. If you did everything correctly, they should be the same for the given $N$.

The other system parameters are:
- $\Delta t = 0.1\text{ [s]}$
- $J = 0.5\text{ [kg }m^2]$
- $B = 0.75\text{ }[\frac{N s}{m}]$

> Hint: You can also use the "display()" method on the lifted system to inspect your results numerically.

> Extra Hint: Check out the next cell to see if there is any function that could help you with this task.

In [ ]:
from helpers import impulse, system_response_to, system_matrices_for, plot_responses


Δt = 0.1
N = 100

# {% if not exercise['3.2.2'].solution %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")

# When you calculate the impulse response using the lifted system, save it in the variable `y`.
y = ...
# {% else %}
# TODO: Review me!
# {{ exercise['3.2.2'].comment }}
A, b, c = system_matrices_for(Δt=Δt, J=0.5, B=0.75)
x_0 = np.array([0, 0])

lifted_system = LiftedSystem.of(A=A, b=b, c=c, N=N, x_0=x_0)
lifted_system.display()

u = impulse(N)
y = lifted_system.response_to(u, shift=True)
# {% endif %}

# To compensate for the shift of size m from the lifted system, we calculate a bit more of
# the system response "manually".
u = impulse(N + lifted_system.m)
y_d = system_response_to(u=u, A=A, b=b, c=c, x_0=x_0)

# We compare the responses here
_ = plot_responses(y, y_d, Δt=Δt)


### 3.3 Reflection

After constructing and visualizing the lifted system matrix, consider the following questions:

1. What does the structure of the $\mathbf{G}$ matrix tell you about the system's behavior?
2. How does an impulse (torque applied instantaneously) affect the system's output (angle of the second joint) over time?
3. Can you identify any patterns in the $\mathbf{G}$ matrix that relate to the original $A$, $b$, and $c$ matrices?

## 4. Iterative Learning Control Law

In this section, we'll introduce the Iterative Learning Control (ILC) law that we'll use to improve the tracking performance of our robot arm over multiple iterations.

### 4.1 The ILC Update Law

The basic idea of ILC is to use information from previous iterations to improve the control input for the next iteration. We'll use the following law:

$u_{j+1} = Q (u_j + L e_j)$

Where:
- $u_j$ is the input sequence for iteration $j$.
- $e_j = y_d - y_j$ is the error in the angle for iteration $j$.
- $L$ is the learning gain matrix.
- $Q$ is the "filter" matrix.
- $y_d$ is the ideal $\theta_2$ sequence.

### 4.2 Designing the ILC

You've learned several approaches to designing the ILC law. In this assignment, we'll be using the **norm-optimal ILC** design method, which is a bit more complex
than the *P-type* or *PD-type* designs. We first express the $Q$ and $L$ matrices with the positive definite matrices $V$, $S$, and $R$:
- $Q = (G^T V G + S + R)^{-1} (G^T V G + R)$  
- $L = (G^T V G + R)^{-1} G^T V$

Where:
- $G$ is the lifted system matrix.

Furthermore, we restrict ourselves to three hyperparameters to express the positive definite matrices:
- $V = v I > 0$
- $S = s I > 0$
- $R = r I > 0$

### 4.3 Algorithm

The ILC algorithm would look something like this:

1. Initialize $u_0$ (often set to zero for all time steps)
2. For iterations $j = 0, 1, 2, ...$, until a termination condition is met:  
   a. Apply $u_j$ to the system and measure the output $y_j$  
   b. Calculate the error $e_j = y_d - y_j$  
   c. Update the input based on the ILC control law.

#### Exercise 4.3.1

Implement the function `ilc(lifted_system, y_d, u, v, s, r, max_iterations, disturbance)` that performs at most `max_iterations` iterations of the ILC algorithm and returns the learned control signal $u$. The function should take the following inputs:
- The lifted system description `lifted_system`.
- The desired output sequence `y_d`.
- The initial input sequence `u`.
- The learning hyperparameters `v`, `s`, and `r`.
- The maximum number of iterations `max_iterations`.
- A function `disturbance(j, u)` that returns the disturbance vector $\mathbf{v_j}$ for iteration $j$ and input $\mathbf{u_j}$.

The function also has a `on_iteration` parameter, which is a function that should be called at the end of each iteration. This will be useful for plotting the progress of the ILC algorithm later on. All you have to do is to make
sure to call it after every iteration with the correct arguments:
- `iteration`: The current iteration number.
- `y`: The output sequence for the current iteration.
- `y_d`: The desired output sequence.

> Hint: You can use the `N` attribute of the `lifted_system` object to get the proper horizon for the ILC algorithm, like so: `N = lifted_system.N`.

In [ ]:
from helpers import Disturbance, IterationCallback, no_callback, no_disturbance


# {% if exercise['4.3.1'].solution %}
def ilc_matrices_for(
    lifted_system: LiftedSystem, *, v: float, s: float, r: float
) -> tuple[Matrix, Matrix]:
    N = lifted_system.N
    G = lifted_system.G
    V = np.eye(N) * v
    S = np.eye(N) * s
    R = np.eye(N) * r

    Q = np.linalg.solve(G.T @ V @ G + S + R, G.T @ V @ G + R)
    L = np.linalg.solve(G.T @ V @ G + R, G.T @ V)

    return Q, L


def ilc_step(
    lifted_system: LiftedSystem,
    *,
    y_d: Vector,
    u: Vector,
    Q: Matrix,
    L: Matrix,
    disturbance: Vector,
) -> tuple[Vector, Vector]:
    y = lifted_system.response_to(u, v=disturbance)
    e = y_d - y

    return Q @ (u + L @ e), y


# {% endif %}
def ilc(
    lifted_system: LiftedSystem,
    *,
    y_d: Vector,
    u: Vector,
    v: float,
    s: float,
    r: float,
    max_iterations: int,
    disturbance: Disturbance = no_disturbance,
    on_iteration: IterationCallback = no_callback,
) -> Vector:
    # {% if exercise['4.3.1'].solution %}
    # TODO: Review me!
    # We calculate the matrices Q and L first.
    Q, L = ilc_matrices_for(lifted_system, v=v, s=s, r=r)

    # Now we run the ILC algorithm, iterate and update the input u.
    for j in range(max_iterations):
        u, y = ilc_step(
            lifted_system, y_d=y_d, u=u, Q=Q, L=L, disturbance=disturbance(j, u)
        )
        on_iteration(j, y=y, y_d=y_d)

    return u
    # {% else %}
    # TODO: Implement me!
    raise NotImplementedError("Not implemented!")

    # You could start by calculating the matrices Q and L first. `np.eye(N)` can help
    # with creating the identity matrix.

    # Then you can move onto implementing the ILC algorithm.

    # At the end of each iteration, don't forget to call the provided `on_iteration`
    # function. It's needed to make our animations work.
    # {% endif %}

#### Exercise 4.3.2

Before we see how the ILC algorithm performs on a robot arm, let's examine how the output sequence $y$ changes to get closer to the desired output sequence $y_d$.  
The function `animate_ilc(lifted_system, y_d, u, v, s, r, max_iterations)` is already provided for you. It uses the `ilc` function you implemented and plots the
resulting output sequence for each iteration.

Use this function to see how ILC works for a desired output sequence $y_d = e^{\frac{-(k - \mu)^2}{2\sigma^2}}$, where $\mu = N/2$ and $\sigma = N/10$ and $k$ is the time step.
The system parameters can be taken from [Exercise 3.2.2](#Exercise-3.2.2). 

The learning hyperparameters can be set to:
- $v = 150$
- $s = 1$
- $r = 1500$

*Note: these are not typical values, they just give a good animation.*

Perform $200$ iterations and observe the results.

> Reminders: 
>
> - The function `system_matrices_for(Δt, J, B)` is provided and can be used to generate the correct system matrices.
> - The starting state of the system - $x_0$ - can be set to $[0, 0]$.

In [ ]:
from helpers import animate_ilc as ilc_animation_helper


def animate_ilc(
    lifted_system: LiftedSystem,
    *,
    y_d: Vector,
    u: Vector,
    v: float,
    s: float,
    r: float,
    max_iterations: int,
    Δt: float,
) -> None:
    """Animates the ILC algorithm for the given system and parameters.

    Args:
        lifted_system: The lifted system to use for the ILC algorithm.
        y_d: The desired output trajectory.
        u: The initial input trajectory.
        v: The disturbance weight.
        s: The input weight.
        r: The output weight.
        max_iterations: The maximum number of iterations to run the ILC algorithm.
        Δt: The time step of the system.

    Example:
        To animate the ILC algorithm for a lifted system `lifted_system` with a desired output
        trajectory `y_d` and an initial input trajectory `u`, you can use the following code:

        ```python
        u = ...  # Some initial input trajectory, like all zeros
        y_d = ...  # Some desired output trajectory
        lifted_system = ...  # The lifted system to use
        animate_ilc(lifted_system, y_d=y_d, u=u, v=0.1, s=0.1, r=0.1, max_iterations=100, Δt=0.1)
        ```

        This will animate the ILC algorithm for the given system and parameters.
    """
    ilc_animation_helper(
        lifted_system=lifted_system,
        ilc=ilc,
        y_d=y_d,
        u=u,
        v=v,
        s=s,
        r=r,
        max_iterations=max_iterations,
        Δt=Δt,
    )

In [ ]:
Δt = 0.1
N = 100

# {% if exercise['4.3.2'].solution %}
# TODO: Review me!
A, b, c = system_matrices_for(Δt=Δt, J=0.5, B=0.75)
x_0 = np.array([0, 0])

lifted_system = LiftedSystem.of(A=A, b=b, c=c, N=N, x_0=x_0)

# The desired signal is a gaussian bell curve
mean = N / 2
std_dev = N / 10
t = np.linspace(0, N - 1, N)
y_d = np.exp(-0.5 * ((t - mean) / std_dev) ** 2)

# The initial control signal can just be a zero vector
u = np.zeros(N)

animate_ilc(lifted_system, y_d=y_d, u=u, v=150, s=1, r=1500, max_iterations=200, Δt=Δt)
# {% else %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")

# First, start by creating the system matrices A, b, and c, as well as the initial state x_0.

# Then, create the lifted system object.

# Now you can define the desired signal y_d (e.g. a gaussian bell curve).

# Finally, run the provided `animate_ilc` function with the appropriate parameters (what should the initial control signal be?).
# {% endif %}

#### Exercise 4.3.3

Now is the time to test the ILC algorithm on our robot arm. To start simple, let's try to make the end-effector trace a straight line from $(-1, 1)$ to $(1, 1)$.
The ideal $\theta_2$ sequence is already calculated for you, using inverse kinematics. You can use the `animate` function to visualize the robot arm's movement.

Some other things you will need:
- The system parameters $\Delta t$, $J$, and $B$, as well as the sample size $N$ can be used from the previous section.
- The initial state of the robot arm $x_0$ should be such that the arm starts on the path you want to trace.
- The initial input sequence $u_0$ can also be set to zero for all time steps.
- Assume no disturbance for now.
- The maximum number of iterations can be set to $50$.
- The learning hyperparameters can be set as follows:
    - $v = 1$
    - $s = 0.1$
    - $r = 0.1$

Implement the general function `learn_torque_for_trajectory(robot, trajectory, ...)`, which will take the robot arm, the desired trajectory, and any other parameters you might need. This function should:
- Calculate the ideal $\theta_2$ sequence for the desired trajectory and visualize the ideal end-effector movement (already implemented).
- Learn the control signal using ILC and visualize the learned end-effector movement.

You can then use this function to make the end-effector trace the straight line from $(-1, 1)$ to $(1, 1)$.

If you use the `show_ideal_trajectory` parameter to show the ideal trajectory only when asked, your outputs later on will be less cluttered.

> Hint: Use the `animate` function to visualize the robot arm's movement. Use `linspace` to generate the desired trajectory once `learn_torque_for_trajectory` is implemented.

In [ ]:
def learn_torque_for_trajectory(
    robot: RobotArm,
    trajectory: list[Point],
    starting_guess: JointAngles,
    *,
    Δt: float,
    J: float,
    B: float,
    v: float,
    s: float,
    r: float,
    max_iterations: int,
    disturbance: Disturbance = no_disturbance,
    show_ideal_trajectory: bool = True,
) -> None:
    # Calculate the desired joint angles
    joint_angles = trajectory_joint_angles_for(
        robot, trajectory, starting_guess=starting_guess
    )

    if show_ideal_trajectory:
        animate(
            robot,
            joint_angles=joint_angles,
            align_starting_position=True,
            subtitle="(Ideal Trajectory)",
        )

    # {% if exercise['4.3.3'].solution %}
    # TODO: Review me!

    # Define the system matrices and create the lifted system
    N = len(joint_angles)
    A, b, c = system_matrices_for(Δt=Δt, J=J, B=B)

    # Initial theta_2 is that of the first joint angles, and the initial angular velocity is 0
    x_0 = np.array([joint_angles[0][1], 0])
    lifted_system = LiftedSystem.of(A=A, b=b, c=c, N=N, x_0=x_0)

    # Define the desired output and calculate the control input
    y_d = np.array([theta_2 for _, theta_2 in joint_angles])
    u = ilc(
        lifted_system,
        y_d=y_d,
        u=np.zeros(N),
        v=v,
        s=s,
        r=r,
        max_iterations=max_iterations,
        disturbance=disturbance,
    )

    # Calculate the joint angles resulting from the learned control input and animate it
    y = lifted_system.response_to(u)
    theta_1s = [theta_1 for theta_1, _ in joint_angles]
    learned_joint_angles = JointAngles.combining(theta_1s, y)

    animate(
        robot,
        joint_angles=learned_joint_angles,
        align_starting_position=True,
        subtitle="(Learned Trajectory)",
    )
    # {% else %}
    # TODO: Implement me!

    # You can start with creating the lifted system.
    # Hint: think about the initial state x_0. It should not be a zero vector.

    # Once you use your lifted system with the ILC algorithm, you can calculate the joint angles resulting from the learned control input.
    # Hint: Since we are learning the torque for the second joint only, you can take the joint angles for the first joint from the trajectory.

    raise NotImplementedError("Not implemented!")
    # {% endif %}

In [ ]:
roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=0, theta_2=0)

# {% if exercise['4.3.3'].solution %}
# TODO: Review me!
line_trajectory: list[Point] = [Point(x, 1) for x in linspace(-1, 1, 100)]
learn_torque_for_trajectory(
    roberto,
    line_trajectory,
    starting_guess=JointAngles(π, -π / 2),
    Δt=0.1,
    J=0.5,
    B=0.75,
    v=1,
    s=0.1,
    r=0.1,
    max_iterations=50,
)
# {% else %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")
# {% endif %}

#### Exercise 4.3.4

Now try out the following trajectories:  

a) A square path: $(1, 0) \rightarrow (1, 1) \rightarrow (-1, 1) \rightarrow (-1, -1) \rightarrow (1, -1) \rightarrow (1, 0)$  
b) A circular path with radius $0.5$ centered at $(0, 0)$.  
c) A figure 8 path somewhere right of the origin.  
d) The wiggly diamond path from before.

> Hint: Parameterized curves are your friend!

> Another Hint: Use $sin(2t)$ and $sin(t)$, along with some shift to the right or left to generate the figure 8 path.

In [ ]:
from typing import Any

roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=0, theta_2=0)

# {% if exercise['4.3.4'].solution %}
# TODO: Review me!
# {{ exercise['4.3.4'].comment }}
parameters: dict[str, Any] = dict(
    Δt=0.1, J=0.5, B=0.75, v=1, s=0.1, r=0.1, max_iterations=50
)

print("a) Square")
learn_torque_for_trajectory(
    roberto,
    trajectory=(
        [Point(1, y) for y in linspace(0, 1, 25)]
        + [Point(x, 1) for x in linspace(1, -1, 50)]
        + [Point(-1, y) for y in linspace(1, -1, 50)]
        + [Point(x, -1) for x in linspace(-1, 1, 50)]
        + [Point(1, y) for y in linspace(-1, 0, 25)]
    ),
    starting_guess=JointAngles(-π / 4, π / 4),
    **parameters,
)

print("b) Circle")
learn_torque_for_trajectory(
    roberto,
    trajectory=[
        Point(1 / 2 * np.cos(t), 1 / 2 * np.sin(t)) for t in linspace(0, 2 * π, 100)
    ],
    starting_guess=JointAngles(0, π),
    **parameters,
)

print("c) Figure 8")
learn_torque_for_trajectory(
    roberto,
    trajectory=[
        Point(1 + 1 / 2 * np.sin(2 * t), 1 / 2 * np.sin(t))
        for t in linspace(0, 4 * π, 100)
    ],
    starting_guess=JointAngles(-π / 4, π / 4),
    **parameters,
)

print("d) Wiggly Diamond")
learn_torque_for_trajectory(
    roberto,
    trajectory=(
        [Point(1 - t, t + 1 / 4 * sin(4 * π * t)) for t in linspace(0, 1, 100)]
        + [Point(t - 1, t + 1 / 4 * sin(4 * π * t)) for t in linspace(1, 0, 100)]
        + [Point(t - 1, -t - 1 / 4 * sin(4 * π * t)) for t in linspace(0, 1, 100)]
        + [Point(1 - t, -t - 1 / 4 * sin(4 * π * t)) for t in linspace(1, 0, 100)]
    ),
    starting_guess=JointAngles(-π / 4, π / 4),
    **parameters,
)
# {% else %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")
# {% endif %}

#### Exercise 4.3.5

Finally, let's see how well ILC can learn to trace a trajectory, when the system is subject to disturbances. Create the following disturbance functions:  

a) A constant disturbance of $0.25$.  
b) A sinusoidal disturbance with amplitude $1$ and a frequency of $60$ Hz.  
c) Gaussian noise with an amplitude of $5$, mean $0$ and standard deviation $0.1$.

For each disturbance, apply it to the system and see how well ILC can learn to trace the square path from before. It would make sense to reuse the `learn_torque_for_trajectory` function to do this,
especially since you can pass the disturbance function as a parameter.

> Hint: You can use the `np.random.normal` function to generate Gaussian noise.

> Advanced Hint: You may want to avoid generating the disturbance at every iteration (unless the disturbance is random). You can generate the disturbance once for a given size and then use it for all iterations.

In [22]:
# You can implement the different disturbance functions/classes here.
# {% if exercise['4.3.5-prep'].solution %}
# TODO: Review me!
# {{ exercise['4.3.5-prep'].comment }}


@dataclass(frozen=True)
class ConstantDisturbance:
    # Generating the constant vector every time is unnecessary, so we cache it
    cache: dict[int, Vector] = field(default_factory=dict, init=False)

    def __call__(self, j: int, u: Vector) -> Vector:
        if len(u) not in self.cache:
            self.cache[len(u)] = np.array([0.25 for _ in range(len(u))])

        return self.cache[len(u)]


@dataclass(frozen=True)
class SinusoidalDisturbance:
    # Generating the sinusoidal vector every time is kind of slow, so we cache it
    cache: dict[int, Vector] = field(default_factory=dict, init=False)

    def __call__(self, j: int, u: Vector) -> Vector:
        if len(u) not in self.cache:
            self.cache[len(u)] = np.array(
                [1.0 * np.sin(2 * π * 0.1 * k) for k in range(len(u))]
            )

        return self.cache[len(u)]


class GaussianNoiseDisturbance:
    # We don't cache the noise, since it's different every time, but we set the seed for reproducibility
    def __init__(self, seed: int = 42) -> None:
        np.random.seed(seed)

    def __call__(self, j: int, u: Vector) -> Vector:
        return 5.0 * np.random.normal(0, 0.1, len(u))


# {% else %}
# TODO: Implement me!
# {% endif %}

In [ ]:
roberto = RobotArm(name="Roberto", l_1=1, l_2=0.5, theta_1=0, theta_2=0)

# {% if exercise['4.3.5'].solution %}
# TODO: Review me!
parameters: dict[str, Any] = dict(
    trajectory=(
        [Point(1, y) for y in linspace(0, 1, 25)]
        + [Point(x, 1) for x in linspace(1, -1, 50)]
        + [Point(-1, y) for y in linspace(1, -1, 50)]
        + [Point(x, -1) for x in linspace(-1, 1, 50)]
        + [Point(1, y) for y in linspace(-1, 0, 25)]
    ),
    starting_guess=JointAngles(-π / 4, π / 4),
    Δt=0.1,
    J=0.5,
    B=0.75,
    v=1,
    s=0.1,
    r=0.1,
    max_iterations=50,
    show_ideal_trajectory=False,
)

print("a) Square with constant disturbance")
learn_torque_for_trajectory(roberto, disturbance=ConstantDisturbance(), **parameters)

print("b) Square with sinusoidal disturbance")
learn_torque_for_trajectory(roberto, disturbance=SinusoidalDisturbance(), **parameters)

print("c) Square with Gaussian noise disturbance")
learn_torque_for_trajectory(
    roberto, disturbance=GaussianNoiseDisturbance(), **parameters
)
# {% else %}
# TODO: Implement me!
print("Oops! Looks like you forgot to implement this cell.")
# {% endif %}

### 4.4 Reflection

Consider the following questions:

1. How do the hyperparameters $v$, $s$, and $r$ affect the learning process?
2. What happens if you increase or decrease the maximum number of iterations?
3. Are all patterns of movement equally easy to learn with ILC? Why or why not?
4. Why do the different disturbances affect the learning process differently?